In [1]:
import csv
from edgeDetection import *
import os
import matplotlib
# matplotlib.use('PS')
import matplotlib.pyplot as plt
import numpy as np
from readTabTxtFile import *
from calTempResolution import *
from trimmedTriggerTime import *
import seaborn as sn
import scipy
from caiman.source_extraction.cnmf.cnmf import load_CNMF
import csv
import pickle

2024-02-26 10:40:02.827550: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from sklearn.cluster import AgglomerativeClustering

In [3]:
directory = "/media/watson/UbuntuHDD/feng_Xin/Xin/Miniscope/5133307132023/reward_seeking/days_with_miniscope_recording/day26/15_18_15"

if not os.path.exists(directory+os.sep+'cellTraces_norm.txt'):
    cnmf_save = load_CNMF(directory + os.sep+'output_rescaled_cnmf_save.hdf5')
    cellTraces = cnmf_save.estimates.C
    valid_idx = cnmf_save.estimates.idx_components
    traces_sz = cellTraces.shape

    with open(directory + os.sep + "cellTraces.txt", 'w', newline="\n") as f:
        f_writer = csv.writer(f, delimiter='\t')
        for i in range(traces_sz[0]):
            f_writer.writerow(cellTraces[i, :])
    with open(directory+os.sep+"validIdx.txt", 'w', newline="\n") as f:
        f_writer = csv.writer(f, delimiter='\t')
        f_writer.writerow(valid_idx)

# for 49140 and 49142
# frameInput = 0
# lickTrigNum = 1
# pokeTrigNum = 2
# caTimeNum = 0
# pumpTrigNum = 4

# for 51331, 51332 and 51333
frameInput = 3
lickTrigNum = 2
pokeTrigNum = 0
caTimeNum = 3
pumpTrigNum = 1

validIdFile = "validIdx.txt"
cellTracesFile = "cellTraces_norm.txt"
pokeTriggerFile = "stim"+str(pokeTrigNum)+".txt"
lickTriggerFile = "stim"+str(lickTrigNum)+".txt"
pumpTriggerFile = "stim"+str(pumpTrigNum)+".txt"

caTimeFile = "stim"+str(caTimeNum)+".txt"
plotSavingFolder = 'Plots'
videoFrameNum = 1000

with open(directory + os.sep + 'frame_correction_pos.pickle','rb') as f:
    frame_correction = pickle.load(f)

def findClosestTimeIndex(t,tSeries):
    tDiff = [abs(t-i) for i in tSeries]
    nd_tDiff = np.array(tDiff)
    pos = np.argmin(nd_tDiff)
    return pos
def findClosestLickIndex(t1Series,refT,tSeries):
    tDiff = [[i-refT,i] for i in t1Series if i-refT>=0]
    nd_tDiff = np.array(tDiff)
    lick_pos = np.argmin(nd_tDiff[:,0])
    pos = findClosestTimeIndex(nd_tDiff[lick_pos,1],tSeries)
    return pos

def findAfterClosestIndex(t1Series,refT,tSeries):
    tDiff = [[i-refT,i] for i in t1Series if i-refT>=0]
    nd_tDiff = np.array(tDiff)
    lick_pos = np.argmin(nd_tDiff[:,0])
    pos = findClosestTimeIndex(nd_tDiff[lick_pos,1],tSeries)
    return pos

def findBeforeClosestIndex(t1Series,refT,tSeries):
    tDiff = [[i-refT,i] for i in t1Series if i-refT<=0]
    nd_tDiff = np.array(tDiff)
    lick_pos = np.argmax(nd_tDiff[:,0])
    pos = findClosestTimeIndex(nd_tDiff[lick_pos,1],tSeries)
    return pos

def findAllLickIndices(t1Series,refT,tSeries,window):
    pos=[]
    tDiff = [[i - tSeries[refT], i] for i in t1Series if i-tSeries[refT]>0 and i-tSeries[refT]-window*caFrameDur<0]
    nd_tDiff = np.array(tDiff)
    for i in nd_tDiff:
        pos_temp = findClosestTimeIndex(i[1],tSeries)
        pos.append(pos_temp)
    return pos

def interpolateNaN(x):
    loc = np.argwhere(np.isnan(x))
    for i in loc:
        x[i[0]] = (x[i[0]-1] + x[i[0]+1])/2
    return x
with open(directory+os.sep+cellTracesFile, "r", newline="\n") as readFile:
    data_reader = readFile.readlines()
# with open(directory+os.sep+validIdFile, "r", newline="") as readFile:
#     id_reader = readFile.readlines()
# with open(directory+os.sep+triggerFile, "r", newline="") as readFile:
#     trig_reader = readFile.readlines()
# with open(directory+os.sep+caTimeFile, "r", newline="") as readFile:
#     caTime_reader = readFile.readlines()

caData = readTabTxtFile(directory+os.sep+cellTracesFile, 'float')
valid_idx = readTabTxtFile(directory+os.sep+validIdFile, 'int')
pokeTrigTime = readTabTxtFile(directory+os.sep+pokeTriggerFile, 'float')
lickTrigTime = readTabTxtFile(directory+os.sep+lickTriggerFile, 'float')
caTime = readTabTxtFile(directory+os.sep+caTimeFile, 'float')
caFrameDur = calTempResolution(caTime)
pumpTrigTime = readTabTxtFile(directory+os.sep+pumpTriggerFile, 'float')

# valid_idx = [int(i) for i in id_reader[0].strip().split("\t")]
# caTime = [float(i) for i in caTime_reader[0].strip().split("\t")]
# trigTime = [float(i) for i in trig_reader[0].strip().split("\t")]
numVideoShift = 0
preTrigWindow = -100
postTrigWindow = 100
len_window = postTrigWindow-preTrigWindow
trimmed_pokeTrigTime = trimmedTriggerTime(pokeTrigTime,caTime,videoFrameNum*numVideoShift,-1)
trimmed_lickTrigTime = trimmedTriggerTime(lickTrigTime,caTime,videoFrameNum*numVideoShift,-1)
trimmed_pumpTrigTime = trimmedTriggerTime(pumpTrigTime,caTime,videoFrameNum*numVideoShift,-1)

# numTrig = len(trimmed_pokeTrigTime)
numTrig = len(trimmed_pumpTrigTime)

numData = len(caData)
numAnalysisWindow = -1*preTrigWindow+postTrigWindow
pokeTimeAdj = 0.03

mean_zscore_ca = np.zeros((len(valid_idx),len_window))
zscore_ca_lick = np.array([])


In [4]:
skip_id = []
for i in range(len(valid_idx)):
    print(i)
    ca_trace = [float(k) for k in data_reader[valid_idx[i]].strip().split("\t")]
    if frame_correction:
        ca_trace_corrected = ca_trace[0:frame_correction[0]+1]+ [float('NaN')]*frame_correction[1] + ca_trace[frame_correction[0]+1:-1]
    else:
        ca_trace_corrected = ca_trace
    ca_raw_traces = []

    for trig_id in range(numTrig):
        try:

            ca_pos_temp_pump = findClosestTimeIndex(trimmed_pumpTrigTime[trig_id], caTime)
            ca_pos_temp = findBeforeClosestIndex(trimmed_pokeTrigTime, trimmed_pumpTrigTime[trig_id], caTime)
            ca_pos_temp_lick = findAfterClosestIndex(trimmed_lickTrigTime, caTime[ca_pos_temp], caTime)

            ca_pos = ca_pos_temp - videoFrameNum * numVideoShift
            ca_pos_lick = ca_pos_temp_lick - videoFrameNum * numVideoShift
            ca_pos_pump = ca_pos_temp_pump - videoFrameNum * numVideoShift


            plot_ca = interpolateNaN(ca_trace_corrected[ca_pos_lick + preTrigWindow: ca_pos_lick + postTrigWindow])
            if np.any(np.isnan(scipy.stats.zscore(plot_ca))):
                print('there is an NaN','current i is:',i,'current trig id is:',trig_id,sep='')
                skip_id.append(trig_id)
                print(skip_id)
                continue
            else:
                print("passed")

        except:
            print("Analysis window is outside ca2+ data size")


0
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
there is an NaNcurrent i is:0current trig id is:16
[16]
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
1
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
2
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed


In [5]:
for i in range(len(valid_idx)):
    print(i)
    ca_trace = [float(k) for k in data_reader[valid_idx[i]].strip().split("\t")]
    if frame_correction:
        ca_trace_corrected = ca_trace[0:frame_correction[0]+1]+ [float('NaN')]*frame_correction[1] + ca_trace[frame_correction[0]+1:-1]
    else:
        ca_trace_corrected = ca_trace
    ca_raw_traces = []

    for trig_id in range(numTrig):
        if trig_id in skip_id:
            continue
        else:
            try:
    
                ca_pos_temp_pump = findClosestTimeIndex(trimmed_pumpTrigTime[trig_id], caTime)
                ca_pos_temp = findBeforeClosestIndex(trimmed_pokeTrigTime, trimmed_pumpTrigTime[trig_id], caTime)
                ca_pos_temp_lick = findAfterClosestIndex(trimmed_lickTrigTime, caTime[ca_pos_temp], caTime)
    
                ca_pos = ca_pos_temp - videoFrameNum * numVideoShift
                ca_pos_lick = ca_pos_temp_lick - videoFrameNum * numVideoShift
                ca_pos_pump = ca_pos_temp_pump - videoFrameNum * numVideoShift
    
    
                plot_ca = interpolateNaN(ca_trace_corrected[ca_pos_lick + preTrigWindow: ca_pos_lick + postTrigWindow])
    
                if len(plot_ca)==abs(preTrigWindow)+abs(postTrigWindow):
                    ca_raw_traces.append(plot_ca)
    
            except:
                print("Analysis window is outside ca2+ data size")
    zscore_ca = scipy.stats.zscore(ca_raw_traces,axis=1,nan_policy='omit')
    if i == 0:
      
        zscore_ca_lick = scipy.stats.zscore(ca_raw_traces,axis=1,nan_policy='omit')
        zscore_ca_lick = np.expand_dims(zscore_ca_lick,axis=0)
    
    else:
        zscore_ca_lick = np.append(zscore_ca_lick,np.expand_dims(scipy.stats.zscore(ca_raw_traces,axis=1,nan_policy='omit'),axis=0),axis=0)

    mean_zscore_ca[i,:] = np.mean(zscore_ca,axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [ ]:
size = zscore_ca_lick.shape
print(size)
print(len(valid_idx))

In [ ]:
lick = mean_zscore_ca

In [ ]:
lick = np.reshape(zscore_ca_lick,(size[0],size[1]*size[2]))
# pca_lick = np.zeros((size[0],size[1],3))

In [ ]:
for i in range(size[0]):
    pca_lick[i,:] = pca.fit_transform(zscore_ca_lick[i,:])

In [ ]:
lick = np.reshape(pca_lick,(size[0],size[1]*3))

In [ ]:
lick.shape

In [ ]:
observed = lick

In [ ]:
from sklearn.decomposition import PCA, KernelPCA, FastICA,SparsePCA
import umap
from sklearn.manifold import TSNE


In [ ]:
pca = PCA(n_components=3)
# pca_k = KernelPCA(n_components=3,kernel="cosine")
# pca_s = SparsePCA(n_components=3,alpha=1)


In [ ]:
X = pca.fit(observed).transform(observed)

In [ ]:
# ica = FastICA(n_components=3,whiten="unit-variance")

In [ ]:
pca.explained_variance_ratio_.cumsum()

In [ ]:
X = umap_embedding 

In [ ]:
clustering = AgglomerativeClustering(n_clusters=3,metric='cosine',linkage='average').fit(X)

In [ ]:
from ipywidgets import interact, FloatSlider

# Define the plotting function
def plot_data(ind_):
    ind = int(ind_)
    plt.figure(figsize=(15,15))
    ax = plt.subplot(331, projection="3d")
    ax.scatter(X[:, 0], X[:, 1],c = clustering.labels_,cmap=plt.cm.viridis, edgecolor="k")
    ax.scatter(X[ind, 0], X[ind, 1],edgecolor="r")
    # ax = plt.subplot(331)
    # ax.scatter(X[:, 0], X[:, 1], c = np.ones(len(valid_idx)),cmap=plt.cm.Greys, edgecolor="k")
    # ax.scatter(X[ind, 0], X[ind, 1], edgecolor="r")
    
    
    plt.subplot(3,3,2)
    plt.plot(mean_zscore_ca[ind,:])
    plt.grid(True)


# Create an interactive slider
interact(plot_data, ind_=FloatSlider(min=0, max=len(valid_idx), step=1, value=1))

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3,max_iter=1000).fit(X)

In [ ]:
# from sklearn.cluster import DBSCAN
# clustering = DBSCAN(eps=4, min_samples=2).fit(X)

In [ ]:
umap_embedding = umap.UMAP(n_components=2).fit_transform(observed)
# tsne_embedding= TSNE().fit_transform(observed)
